# Import libaries and download model

In [ ]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.preprocessing import LabelEncoder

**Segmenter package**

In [6]:
!pip -q install py_vncorenlp
import py_vncorenlp
# save_dir is path_to_vncorenlp
path = '/content/drive/MyDrive/vnese_classification/vncorenlp'
py_vncorenlp.download_model(save_dir=path)
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=path)

Model already exists, skipping download.


Exception: Please download the VnCoreNLP model!

**Import stopwords**

In [ ]:
st_path = '/content/drive/MyDrive/vnese_classification/vnese_stopword.txt' #path to stopwords file
f = open(st_path)
stop_words = set()
for word in f.readlines():
  stop_words.add(word.split('\n')[0])

**Pre-trained model**

In [ ]:
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

# Preprocessing

In [ ]:
import string
import re


def preprocessing(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+\s+', '', text)
    text = re.sub(r'<[^>]*>', '', text)
    text = text.lower()
    text = ''.join(rdrsegmenter.word_segment(text))
    text = text.split(' ')
    text = list(filter(lambda word: word not in stop_words, text))
    return text

In [ ]:
temp = []
label = set()

#open trainning data file
trainning_path = '/content/drive/MyDrive/vnese_classification/trainning.txt'
with open(trainning_path, 'r') as f:
  for line in f:
    if line != '\n':
      temp.append(line.split('\t'))

In [ ]:
trainning = []
maxlen = 0
for i in range(len(temp)):
    text = temp[i][1]
    text = preprocessing(text)
    maxlen = max(maxlen, len(text))
    trainning.append([temp[i][0], ' '.join(text)])
    label.add(temp[i][0])

In [ ]:
trainning [0][1]

'học_sinh học điều_kiện tạm_bợ nlđ học_sinh trường tiểu_học đống_đa p bình_thạnh tphcm gởi học tạm km trường xuống_cấp trầm_trọng kế_hoạch đầu năm_học trường tiểu_học đống_đa khởi_công xây_dựng chỗ học năm_học kết_thúc xây trường phụ_huynh cảm_thấy yên_tâm hằng con_em học tình_trạng trường_lớp tạm_bợ đi_lại kém an_toàn như_thế'

**Label encoding**

In [ ]:
labels_dict = {char: i for i, char in enumerate(label)}

In [ ]:
labels_dict

{'__KD__': 0,
 '__TT__': 1,
 '__DS__': 2,
 '__PL__': 3,
 '__VH__': 4,
 '__KH__': 5,
 '__CTXH__': 6,
 '__TG__': 7,
 '__SK__': 8,
 '__VT__': 9}

In [ ]:
# # lưu lại labels dict chay vì em lười chạy lại notebook từ đầu :))
# labels_dict = {'__VT__': 0,
#  '__TT__': 1,
#  '__KH__': 2,
#  '__DS__': 3,
#  '__SK__': 4,
#  '__KD__': 5,
#  '__TG__': 6,
#  '__VH__': 7,
#  '__CTXH__': 8,
#  '__PL__': 9}

# Embedding data

In [ ]:
import torch
def get_emb_vector(input_ids):
    input_ids  = torch.tensor([input_ids]).to(torch.long)
    with torch.no_grad():
        features = phobert(input_ids)[1].squeeze(0)
    return features

def text2ids(text):
    tkz = tokenizer.encode(text)
    return tkz

In [ ]:
data_len = len(trainning)
features = []
labels = []
MAX_LEN = 256
for i in range(data_len):
    input_ids = text2ids(trainning[i][1])
    for j in range(int(len(input_ids) / 256) + 1):
      segment = input_ids[256 * j : 256 * j + min(256, len(input_ids) - 256 * j)]
      if len(segment) < 5:
          break
      features.append(get_emb_vector(segment))
      labels.append(labels_dict[trainning[i][0]])
    print(i , '/', data_len)

Streaming output truncated to the last 5000 lines.
0 / 5000
1 / 5000
2 / 5000
3 / 5000
4 / 5000
5 / 5000
6 / 5000
7 / 5000
8 / 5000
9 / 5000
10 / 5000
11 / 5000
12 / 5000
13 / 5000
14 / 5000
15 / 5000
16 / 5000
17 / 5000
18 / 5000
19 / 5000
20 / 5000
21 / 5000
22 / 5000
23 / 5000
24 / 5000
25 / 5000
26 / 5000
27 / 5000
28 / 5000
29 / 5000
30 / 5000
31 / 5000
32 / 5000
33 / 5000
34 / 5000
35 / 5000
36 / 5000
37 / 5000
38 / 5000
39 / 5000
40 / 5000
41 / 5000
42 / 5000
43 / 5000
44 / 5000
45 / 5000
46 / 5000
47 / 5000
48 / 5000
49 / 5000
50 / 5000
51 / 5000
52 / 5000
53 / 5000
54 / 5000
55 / 5000
56 / 5000
57 / 5000
58 / 5000
59 / 5000
60 / 5000
61 / 5000
62 / 5000
63 / 5000
64 / 5000
65 / 5000
66 / 5000
67 / 5000
68 / 5000
69 / 5000
70 / 5000
71 / 5000
72 / 5000
73 / 5000
74 / 5000
75 / 5000
76 / 5000
77 / 5000
78 / 5000
79 / 5000
80 / 5000
81 / 5000
82 / 5000
83 / 5000
84 / 5000
85 / 5000
86 / 5000
87 / 5000
88 / 5000
89 / 5000
90 / 5000
91 / 5000
92 / 5000
93 / 5000
94 / 5000
95 / 5000

Chuyển thành tensor

In [ ]:
features = torch.stack(features)
labels = torch.tensor(labels)

Lưu tensor

In [ ]:
f_p = '/content/drive/MyDrive/vnese_classification/features.pth'
l_p = '/content/drive/MyDrive/vnese_classification/labels.pth'

In [ ]:
torch.save(features, f_p)
torch.save(labels, l_p)

Tải lại tensor

In [ ]:
import torch
features = torch.load(f_p)
labels = torch.load(l_p)

# Modeling and trainning

In [ ]:
# chuyển dữ liệu
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(features, labels)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# định nghĩa model
class ClassifierModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(ClassifierModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Định nghĩa các tham số mô hình
input_size = features.shape[1]
hidden_size = 128
output_size = len(labels_dict)  # Số nhãn
dropout_rate = 0.5

model = ClassifierModel(input_size, hidden_size, output_size, dropout_rate)


Định nghĩa hàm loss, optimizer, scheduler

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Giảm tốc độ học khi loss không hội tụ
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

# Cài đặt Early Stopping
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=10, min_delta=0.001)


Train model


In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    val_loss = train_loss
    scheduler.step(val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break


Epoch 1/100, Train Loss: 1.4516, Val Loss: 1.4516
Epoch 2/100, Train Loss: 0.7393, Val Loss: 0.7393
Epoch 3/100, Train Loss: 0.5987, Val Loss: 0.5987
Epoch 4/100, Train Loss: 0.5264, Val Loss: 0.5264
Epoch 5/100, Train Loss: 0.4934, Val Loss: 0.4934
Epoch 6/100, Train Loss: 0.4582, Val Loss: 0.4582
Epoch 7/100, Train Loss: 0.4351, Val Loss: 0.4351
Epoch 8/100, Train Loss: 0.4194, Val Loss: 0.4194
Epoch 9/100, Train Loss: 0.3919, Val Loss: 0.3919
Epoch 10/100, Train Loss: 0.3852, Val Loss: 0.3852
Epoch 11/100, Train Loss: 0.3624, Val Loss: 0.3624
Epoch 12/100, Train Loss: 0.3569, Val Loss: 0.3569
Epoch 13/100, Train Loss: 0.3471, Val Loss: 0.3471
Epoch 14/100, Train Loss: 0.3308, Val Loss: 0.3308
Epoch 15/100, Train Loss: 0.3197, Val Loss: 0.3197
Epoch 16/100, Train Loss: 0.3198, Val Loss: 0.3198
Epoch 17/100, Train Loss: 0.3072, Val Loss: 0.3072
Epoch 18/100, Train Loss: 0.3036, Val Loss: 0.3036
Epoch 19/100, Train Loss: 0.2935, Val Loss: 0.2935
Epoch 20/100, Train Loss: 0.2762, Val Lo

In [ ]:
# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/vnese_classification/classification_model.pth')

# Testing

Xử lý dữ liệu test

In [ ]:
temp = []
with open('/content/drive/MyDrive/vnese_classification/testing.txt', 'r') as f:
  for line in f:
    if line != '\n':
      temp.append(line.strip())

In [ ]:
testing = []
for i in range(len(temp)):
    text = temp[i]
    text = preprocessing(text)
    testing.append(' '.join(text))

In [ ]:
maxSizeTensor = 256
tests = []
for i in range(len(testing)):
  input_ids = torch.tensor([tokenizer.encode(testing[i], truncation=True, max_length=maxSizeTensor)])

  with torch.no_grad():
    tests.append(phobert(input_ids)[1].squeeze())
  print(i, '/', len(testing))

0 / 5000
1 / 5000
2 / 5000
3 / 5000
4 / 5000
5 / 5000
6 / 5000
7 / 5000
8 / 5000
9 / 5000
10 / 5000
11 / 5000
12 / 5000
13 / 5000
14 / 5000
15 / 5000
16 / 5000
17 / 5000
18 / 5000
19 / 5000
20 / 5000
21 / 5000
22 / 5000
23 / 5000
24 / 5000
25 / 5000
26 / 5000
27 / 5000
28 / 5000
29 / 5000
30 / 5000
31 / 5000
32 / 5000
33 / 5000
34 / 5000
35 / 5000
36 / 5000
37 / 5000
38 / 5000
39 / 5000
40 / 5000
41 / 5000
42 / 5000
43 / 5000
44 / 5000
45 / 5000
46 / 5000
47 / 5000
48 / 5000
49 / 5000
50 / 5000
51 / 5000
52 / 5000
53 / 5000
54 / 5000
55 / 5000
56 / 5000
57 / 5000
58 / 5000
59 / 5000
60 / 5000
61 / 5000
62 / 5000
63 / 5000
64 / 5000
65 / 5000
66 / 5000
67 / 5000
68 / 5000
69 / 5000
70 / 5000
71 / 5000
72 / 5000
73 / 5000
74 / 5000
75 / 5000
76 / 5000
77 / 5000
78 / 5000
79 / 5000
80 / 5000
81 / 5000
82 / 5000
83 / 5000
84 / 5000
85 / 5000
86 / 5000
87 / 5000
88 / 5000
89 / 5000
90 / 5000
91 / 5000
92 / 5000
93 / 5000
94 / 5000
95 / 5000
96 / 5000
97 / 5000
98 / 5000
99 / 5000
100 / 5000

In [ ]:
tests = torch.stack(tests)
tests.shape

torch.Size([5000, 768])

In [ ]:
torch.save(tests, '/content/drive/MyDrive/vnese_classification/tests.pth')

In [ ]:
test_features = torch.load('/content/drive/MyDrive/vnese_classification/tests.pth')

In [ ]:
test_features.shape

torch.Size([5000, 768])

In [ ]:
with open('/content/drive/MyDrive/vnese_classification/testing_labels.txt', 'r', encoding='utf-8') as f:
    test_labels = f.readlines()

test_labels = [label.strip() for label in test_labels]

# Chuyển đổi nhãn thành chỉ số
test_labels = [labels_dict[label] for label in test_labels]

SyntaxError: unexpected character after line continuation character (<ipython-input-11-0bb1fbd7e41c>, line 1)

In [ ]:
test_labels

[3,
 3,
 9,
 3,
 5,
 5,
 6,
 5,
 1,
 6,
 8,
 7,
 0,
 3,
 0,
 6,
 4,
 1,
 6,
 7,
 2,
 9,
 3,
 5,
 9,
 6,
 6,
 1,
 3,
 0,
 1,
 7,
 0,
 7,
 9,
 3,
 6,
 0,
 4,
 1,
 3,
 8,
 8,
 5,
 7,
 5,
 1,
 9,
 0,
 4,
 2,
 7,
 5,
 0,
 9,
 4,
 8,
 1,
 9,
 7,
 9,
 5,
 9,
 5,
 3,
 9,
 5,
 9,
 7,
 2,
 0,
 7,
 2,
 8,
 1,
 0,
 9,
 9,
 9,
 4,
 8,
 7,
 0,
 1,
 5,
 3,
 5,
 5,
 5,
 3,
 2,
 3,
 1,
 4,
 0,
 2,
 4,
 3,
 2,
 5,
 1,
 0,
 1,
 7,
 3,
 3,
 6,
 4,
 4,
 3,
 1,
 1,
 3,
 5,
 5,
 0,
 7,
 4,
 1,
 7,
 0,
 7,
 8,
 3,
 3,
 4,
 4,
 4,
 8,
 1,
 9,
 2,
 1,
 8,
 2,
 9,
 8,
 4,
 8,
 5,
 4,
 8,
 8,
 0,
 0,
 9,
 2,
 1,
 1,
 2,
 2,
 3,
 1,
 6,
 7,
 6,
 9,
 7,
 2,
 4,
 0,
 4,
 7,
 5,
 8,
 4,
 3,
 2,
 6,
 3,
 1,
 9,
 2,
 4,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 0,
 3,
 5,
 4,
 2,
 2,
 7,
 2,
 0,
 4,
 4,
 9,
 2,
 2,
 8,
 0,
 9,
 4,
 9,
 9,
 4,
 4,
 1,
 4,
 0,
 7,
 7,
 6,
 1,
 1,
 7,
 1,
 5,
 5,
 5,
 2,
 0,
 3,
 3,
 8,
 5,
 4,
 1,
 7,
 1,
 1,
 9,
 3,
 0,
 6,
 6,
 2,
 7,
 8,
 9,
 3,
 9,
 1,
 4,
 5,
 3,
 9,
 7,
 7,
 5,
 4,
 1,
 2,
 2,


In [ ]:
test_labels = torch.tensor(test_labels)
torch.save(test_labels, '/content/drive/MyDrive/vnese_classification/test_labels.pth')

In [ ]:
test_labels = torch.load('/content/drive/MyDrive/vnese_classification/test_labels.pth')
test_labels.shape

torch.Size([5000])

Đánh giá model

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

model.eval()

# Dự đoán nhãn cho toàn bộ tập dữ liệu test
with torch.no_grad():
    outputs = model(test_features)
    _, predicted = torch.max(outputs, 1)
    correct_predictions = (predicted == test_labels)

precision = precision_score(test_labels, predicted, average='weighted')
recall = recall_score(test_labels, predicted, average='weighted')
f1 = f1_score(test_labels, predicted, average='weighted')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

# Báo cáo chi tiết cho từng nhãn
print(classification_report(test_labels, predicted, target_names=list(labels_dict.keys())))

Precision: 0.8935
Recall: 0.8930
F1-score: 0.8924
              precision    recall  f1-score   support

      __VT__       0.91      0.94      0.92       500
      __TT__       0.98      0.95      0.96       500
      __KH__       0.90      0.83      0.87       500
      __DS__       0.84      0.89      0.86       500
      __SK__       0.89      0.90      0.90       500
      __KD__       0.86      0.89      0.88       500
      __TG__       0.86      0.91      0.88       500
      __VH__       0.95      0.91      0.93       500
    __CTXH__       0.84      0.73      0.78       500
      __PL__       0.91      0.96      0.93       500

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

